In [1]:
import keras
import cv2
from matplotlib import pyplot as plt

2023-09-11 22:13:50.677756: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 22:13:50.879258: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 22:13:50.880375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 22:13:51.752135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

2023-09-11 22:13:53.305721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-11 22:13:53.432168: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def takePhoto():
    cam = cv2.VideoCapture(0)
    ret,frame = cam.read()
    cv2.imwrite('picture.jpg',frame)
    cam.release()

In [4]:
def predict_image(path):
    image = Image.open(path).convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predicts the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    return class_name[2:].replace("\n","")
    #print("Confidence Score:", confidence_score)

In [5]:
def pictureAug():
    image = cv2.imread("picture.jpg")
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(7,7),0)
    thresh =cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    kernal = cv2.getStructuringElement(cv2.MORPH_RECT,(2,1))
    dilate=cv2.dilate(thresh,kernal,iterations=1)
    cnts = cv2.findContours(dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts)==2 else cnts[1]
    cnts= sorted(cnts,key=lambda x: cv2.boundingRect(x)[0])
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        #print(x,y,w,h)
        if h > 100 and w > 100 and h < 1000 and w < 1000:
            roi = image[y:y+h, x:x+w]
            cv2.imwrite("test3.png",roi)
            cv2.rectangle(image,(x,y),(x+w,y+h),(36,255,12),2)
    cv2.imwrite("test4.png",image)
    img = cv2.imread('test3.png')


    h, w, channels = img.shape
    images = []
    height = w//3
    width = h//3
    for i in range(3):
        for j in range(3):
            print("made here")
            images.append(img[(i*height):((i+1)*height), (j*width):((j+1)*width)])


    #cv2.imwrite('left.jpg', left_part)

    filename_base = "image_"
    file_extension = ".png"
    counter = 1

    for image in images:
        # Create the filename with a changing component (counter)
        filename = f"{filename_base}{counter:03d}{file_extension}"

        # Save the image using the generated filename
        cv2.imwrite(filename, image)

        # Increment the counter for the next iteration
        counter += 1

In [6]:
def videoAug(image):
    #image = cv2.imread("picture.jpg")
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(7,7),0)
    thresh =cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    kernal = cv2.getStructuringElement(cv2.MORPH_RECT,(2,1))
    dilate=cv2.dilate(thresh,kernal,iterations=1)
    cnts = cv2.findContours(dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts)==2 else cnts[1]
    cnts= sorted(cnts,key=lambda x: cv2.boundingRect(x)[0])
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        #print(x,y,w,h)
        if h > 100 and w > 100 and h < 1000 and w < 1000:
            #roi = image[y:y+h, x:x+w]
            #cv2.imwrite("test3.png",roi)
            cv2.rectangle(image,(x,y),(x+w,y+h),(36,255,12),2)
    #cv2.imwrite("test4.png",image)
    return image
    

In [7]:
board = {1:' ',2:' ',3:' ',
         4:' ',5:' ',6:' ',
         7:' ',8:' ',9:' '}

def printBoard(board):
    print(board[1]+'|'+board[2]+'|'+board[3])
    print('-+-+-')
    print(board[4]+'|'+board[5]+'|'+board[6])
    print('-+-+-')
    print(board[7]+'|'+board[8]+'|'+board[9])
    print("\n")
    


def spaceIsFree(position):
    if(board[position]==' '):
        return True
    else:
        return False
    
def checkDraw():
    for key in board.keys():
        if board[key]==' ':
            return False
    return True
    
def checkForWin():
    if (board[1] == board[2] and board[1] == board[3] and board[1] != ' '):
        return True
    elif (board[4] == board[5] and board[4] == board[6] and board[4] != ' '):
        return True
    elif (board[7] == board[8] and board[7] == board[9] and board[7] != ' '):
        return True
    elif (board[1] == board[4] and board[1] == board[7] and board[1] != ' '):
        return True
    elif (board[2] == board[5] and board[2] == board[8] and board[2] != ' '):
        return True
    elif (board[3] == board[6] and board[3] == board[9] and board[3] != ' '):
        return True
    elif (board[1] == board[5] and board[1] == board[9] and board[1] != ' '):
        return True
    elif (board[7] == board[5] and board[7] == board[3] and board[7] != ' '):
        return True
    else:
        return False
    
def checkWhichMarkWon(mark):
    if board[1] == board[2] and board[1] == board[3] and board[1] == mark:
        return True
    elif (board[4] == board[5] and board[4] == board[6] and board[4] == mark):
        return True
    elif (board[7] == board[8] and board[7] == board[9] and board[7] == mark):
        return True
    elif (board[1] == board[4] and board[1] == board[7] and board[1] == mark):
        return True
    elif (board[2] == board[5] and board[2] == board[8] and board[2] == mark):
        return True
    elif (board[3] == board[6] and board[3] == board[9] and board[3] == mark):
        return True
    elif (board[1] == board[5] and board[1] == board[9] and board[1] == mark):
        return True
    elif (board[7] == board[5] and board[7] == board[3] and board[7] == mark):
        return True
    else:
        return False
    
def insertLetter(letter,position):
    if spaceIsFree(position):
        board[position] = letter
        printBoard(board)
        if(checkDraw()):
            print("Draw!")
            exit()
            
        if checkForWin():
            if letter =='X':
                print("Bot wins!")
                exit()
            else:
                print("player wins!")
                exit()
        return
    else: 
        print("cant insert there")
        position=int(input("Enter new pos"))
        insertLetter(letter,position)
        return
    
    
player='O'
bot='X'
def playerMove():
    position=int(input("Enter pos"))
    insertLetter(player,position)
    return

def compMove():
    bestScore= -800
    bestMove= 0
    
    for key in board.keys():
        if(board[key]==' '):
            board[key] = bot
            score = minmax(board,False)
            board[key] = ' '
            if(score>bestScore):
                bestScore=score
                bestMove=key
    insertLetter(bot, bestMove)
    return
                
def minmax(board,isMaximizing):
    if checkWhichMarkWon(bot):
        return 1
    elif checkWhichMarkWon(player):
        return -1
    elif checkDraw():
        return 0
    if isMaximizing:
        bestScore=-800

        for key in board.keys():
            if(board[key]==' '):
                board[key] = bot
                score = minmax(board,False)
                board[key] = ' '
                if(score>bestScore):
                    bestScore=score
        return bestScore
    else:
        bestScore=800

        for key in board.keys():
            if(board[key]==' '):
                board[key] = player
                score = minmax(board,True)
                board[key] = ' '
                if(score<bestScore):
                    bestScore=score
        return bestScore
    
                


    

In [8]:
def boardconvert(board):
    
    filename_base = "image_"
    file_extension = ".png"

    for counter in range(1, 10):  # Loop from 1 to 9 (inclusive)

        filename = f"{filename_base}{counter:03d}{file_extension}"
        temp = predict_image(filename)  # Assuming predict_image is defined
        print(temp)
        
        if temp == 'e':
            board[counter] = ' '  # Adjusted index
        else:
            board[counter] = temp  # Adjusted index

    return board

In [ ]:
import cv2
while not checkForWin():
    #photo
    #cam = cv2.VideoCapture(0)
    wait = input('please make your move and press p \n')
    cam = cv2.VideoCapture(0)
    while cam.isOpened():
        ret,frame = cam.read()

        if ret == True:
            cv2.imshow("OJ",videoAug(frame))

            if cv2.waitKey(1) & 0xFF==ord('p'):
                cv2.imwrite('picture.jpg',frame)
                break
    cam.release()
    cv2.destroyAllWindows()
    pictureAug()
    board =  boardconvert(board)
    print("input board")
    printBoard(board)
    #computer move
    print("computer move")
    compMove()

please make your move and press p 
 p


QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread (0x9d77100).
Cannot move to target thread (0x800a000)

QObject::moveToThread: Current thread (0x800a000) is not the object's thread

In [ ]:
import cv2
cam = cv2.VideoCapture(0)

while cam.isOpened():
    ret,frame = cam.read()
    
    if ret == True:
        cv2.imshow("OJ",frame)
        
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
            
cam.release()
cv2.destroyAllWindows()

In [ ]:
# while loop